In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import datetime as dt
import pprint as pp
import pandas as pd
import math
from IPython.display import display
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_pickle('../data/player_game_table.pkl')
teams = pd.read_pickle('../data/team.pkl')
idk = pd.read_pickle('../data/team_game_stats.pkl')
print(idk.team_id.unique())
print(teams)
print(df.columns.values)
print(df.team_id.unique())
df.sort_values('game_date')
print(df.iloc[1].game_date)

ave_features = ['ast','blk','min', 'pts','reb',
                  'stl','to','fantasy_points']
input_features = ['team_against','player_team', 'min_points', 'max_points',
                  'position', 'home_game', 'ave_points_team', 'ave_points_vs_team', 'name']

features = {}
for feature in ave_features:
    features[feature] = []
for feature in input_features:
    features[feature] = []
features['output_feature'] = []
features['season'] = []

In [ ]:
# Create position
from scipy import stats
player_positions = {}
for player_id in df.player_id.unique():
    mode_list =  df.loc[df.player_id == player_id].start_position.tolist()
    mode_list = [x for x in mode_list if x!='']
                        
    temp_mode = stats.mode(mode_list)
    if len(temp_mode[0]) > 0:
        player_positions[player_id] = temp_mode[0][0]
    else:
        player_positions[player_id] = ''

print(player_positions)

In [ ]:
def calculate_team_stats(ind_game, model_features, past_season_games):
    game_box_stats = pd.read_pickle('../data/team_game_stats.pkl')
    overview = pd.read_pickle('../data/game.pkl')
    teams = pd.read_pickle('../data/team.pkl')
    
    team_id = float(ind_game.team_id)
    
    both_team_stats = game_box_stats.loc[game_box_stats.game_id == ind_game.game_id]
    
    vs_team_id = both_team_stats.loc[both_team_stats["team_id"] != team_id]
        
    vs_team_id = vs_team_id['team_id']
    vs_team_id = float(vs_team_id)
    game_ids = past_season_games.loc[past_season_games.team_id == str(team_id)].game_id.unique().tolist()
    vs_game_ids = past_season_games.loc[past_season_games.team_id == str(vs_team_id)].game_id.unique().tolist()
    
    team_id = int(team_id)
    vs_team_id = int(vs_team_id)
    
    
    team_abbrv_row = teams.loc[teams.team_id == str(vs_team_id)]
    team_abbrv = team_abbrv_row.team_abbreviation.values[0]
    model_features['team_against'].append(team_abbrv)
    model_features['player_team'].append(ind_game.team_abbreviation)
    model_features['home_game'].append(True if len(overview.loc[(overview.game_id == ind_game.game_id) & (overview.home_team_id == str(team_id))]) > 0 else False)

    
    pts = game_box_stats.loc[
        (game_box_stats.game_id.isin(game_ids)) & (game_box_stats.team_id == team_id)].pts.tolist()
    vs_pts = game_box_stats.loc[
        (game_box_stats.game_id.isin(vs_game_ids)) & (game_box_stats.team_id == vs_team_id)].pts.tolist()
    
    if (len(pts) > 0):
         model_features['ave_points_team'].append(np.average(pts))
    else:
        print("home")
        print(ind_game.team_abbreviation)
        model_features['ave_points_team'].append(0)
        
    if (len(vs_pts) > 0):
         model_features['ave_points_vs_team'].append(np.average(vs_pts))
    else:
        print("vs")
        print(team_abbrv)
        model_features['ave_points_vs_team'].append(0)
    
    

In [ ]:
def create_nfl_features(df, feats, model_features):
    
   
    rows = df
    rows = rows.sort_values("game_date")
    rows = rows.loc[rows.season.isin(['2017-2018','2016-2017','2015-16'])]
    count = 0
    for i in range(1, len(rows)-1):
        if (not count%100):
            print(count, " out of ", len(rows))
        count += 1
        average_rows = rows.loc[(rows.game_date<rows.iloc[i].game_date) 
                                & (rows.season == rows.iloc[i].season) 
                                & (rows.player_id == rows.iloc[i].player_id)]
        #X[(i - n),0] = pid
        
        
        
        if len(average_rows) > 0:
            
            #averages
            for a,feat in enumerate(feats):
                features[feat].append(np.average(average_rows[feat].tolist()))

            
            calculate_team_stats(rows.iloc[i], model_features, df.loc[(df.game_date<df.iloc[i].game_date) 
                                & (df.season == df.iloc[i].season)])
            model_features["max_points"].append(max(average_rows["fantasy_points"].tolist()))
            model_features["min_points"].append(min(average_rows["fantasy_points"].tolist()))
            model_features["name"].append(rows.iloc[i]["player_name"])
            model_features["season"].append(rows.iloc[i]["season"])
            model_features["position"].append(player_positions[rows.iloc[i]["player_id"]])
            model_features["output_feature"].append(rows.iloc[i]["fantasy_points"])
            
    #max/min
    return model_features

In [ ]:

    
model_features = create_nfl_features(df, ave_features, features)
print(model_features)

with open('nfl_featuers.pkl', 'wb') as fp:
    pickle.dump(features, fp, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X,y)
#X = X[:-1]
#y = y[:-1]
X = X[1:]
y = y[1:]
print(np.shape(X))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=20, random_state=0,n_estimators=10,criterion = 'mae',n_jobs =8,max_features=10)
train_split = 3/4
train_n = int(len(y)*train_split)
regr.fit(X[:train_n],y[:train_n])

plt.plot(list(range(train_n,len(y))),regr.predict(X[train_n:]),'r')
plt.axhline(y = np.average(y))
plt.plot(list(range(train_n,len(y))),y[train_n:],'b')
plt.show()
print(regr.score(X[train_n:],y[train_n:]))